In [1]:
from __future__ import print_function, absolute_import
from WISE_tools import *
import FATS, pandas as pd, numpy as np
from multiprocessing import Pool
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from astropy.modeling.functional_models import Gaussian2D
from astropy.modeling import fitting
from scipy.odr import *
from sklearn.metrics import r2_score

In [2]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

In [3]:
unique_names = parse_source_names('../data/bright_blue_lightcurves/data/')

In [4]:
def KDE_fit(x,y):
    """
    A function that performs a gaussian KDE on x/y data, then fits the result with a sum
    of 2 2D gaussians
    
    Parameters
    ----------
    x : array-like
        x data
    y : array-like
        y data
        
    Returns
    -------
    KDE_bandwidth : float
        Cross-validated kernel bandwidth
    KD_fit_sqresid : float
        The mean squared residual (fit - KDE)^2
    amp_0 : float
        amplitude of largest gaussian
    xmean_0 : float
        x mean of largest gaussian
    ymean_0 : float
        y mean of largest gaussian
    major_std_0 : float
        Standard deviation along major axis of largest gaussian
    theta_0 : float
        Radians between positive X-axis and the major axis of the largest gaussian
    ecc_0 : float
        Eccentricity of an ellipse, taken at an arbitrary height of the largest gaussian
    amp_1 : float
        amplitude of smallest gaussian
    xmean_1 : float
        x mean of smallest gaussian
    ymean_1 : float
        y mean of smallest gaussian
    major_std_1 : float
        Standard deviation along major axis of smallest gaussian
    theta_1 : float
        Radians between positive X-axis and the major axis of the largest gaussian
    ecc_1 : float
        Eccentricity of an ellipse, taken at an arbitrary height of the smallest gaussian
    """
    
    data = np.vstack([x, y]).T
    #Grid search for best KDE bandwidth
    params = {'bandwidth': np.linspace(np.min(np.diff(y)),np.max(np.diff(y)),100)}
    grid = GridSearchCV(KernelDensity(), params)
    grid.fit(data)
    
    KDE_bandwidth = grid.best_estimator_.bandwidth
    
    kde = grid.best_estimator_
    X, Y = np.meshgrid(np.linspace(np.min(x),np.max(x),100), np.linspace(np.min(y),np.max(y),100))

    xy = np.vstack([X.ravel(), Y.ravel()]).T
    #compute the KDE on a 100x100 grid of points
    Z = np.exp(kde.score_samples(xy)).reshape(X.shape)
    
    #fit KDE estimation with 2 Gaussian model
    g2D_init1 = Gaussian2D(amplitude=np.max(Z), x_mean=X[np.unravel_index(np.argmax(Z),Z.shape)], y_mean=Y[np.unravel_index(np.argmax(Z),Z.shape)], x_stddev=np.std(x), y_stddev=np.std(y), theta=0, bounds={'theta': (0,np.pi),'x_mean': (np.min(x),np.max(x)),'y_mean': (np.min(y),np.max(y)),'x_stddev':(0.001,1),'y_stddev':(0.001,1)})
    g2D_init2 = Gaussian2D(amplitude=np.median(Z), x_mean=np.median(x), y_mean=np.median(y), x_stddev=np.std(x), y_stddev=np.std(y), theta=0, bounds={'theta': (0,np.pi),'x_mean': (np.min(x),np.max(x)),'y_mean': (np.min(y),np.max(y)),'x_stddev':(0.001,1),'y_stddev':(0.001,1)})
    g2D_init = g2D_init1 + g2D_init2

    fitter = fitting.LevMarLSQFitter()
    
    g2D = fitter(g2D_init, X, Y, Z)
    
    KD_fit_sqresid = np.mean(np.power(Z-g2D(X,Y),2.0))
    
    #Sort by largest and smallest amplitude gaussian
    i_large = np.argmax([g2D.amplitude_0,g2D.amplitude_1])
    i_small = np.argmin([g2D.amplitude_0,g2D.amplitude_1])
    g2D_large = g2D[i_large]
    g2D_small = g2D[i_small]
    
    amp_0 = g2D_large.amplitude.value
    amp_1 = g2D_small.amplitude.value
    
    xmean_0 = g2D_large.x_mean.value
    xmean_1 = g2D_small.x_mean.value
    
    ymean_0 = g2D_large.y_mean.value
    ymean_1 = g2D_small.y_mean.value
    
    if g2D_large.x_stddev >= g2D_large.y_stddev:
        
        major_std_0 = g2D_large.x_stddev.value
        theta_0 = g2D_large.theta.value
        ecc_0 = np.sqrt(1.0 - (g2D_large.y_stddev.value/g2D_large.x_stddev.value)**2.0)
    
    else:
        
        major_std_0 = g2D_large.y_stddev.value
        
        if g2D_large.theta <= np.pi/2:
            theta_0 = np.pi/2 + g2D_large.theta.value
            
        elif g2D_large.theta > np.pi/2:
            theta_0 = g2D_large.theta.value - np.pi/2
             
        ecc_0 = np.sqrt(1.0 - (g2D_large.x_stddev.value/g2D_large.y_stddev.value)**2.0)
        
    if g2D_small.x_stddev >= g2D_small.y_stddev:
        
        major_std_1 = g2D_small.x_stddev.value
        theta_1 = g2D_small.theta.value
        ecc_1 = np.sqrt(1.0 - (g2D_small.y_stddev.value/g2D_small.x_stddev.value)**2.0)
    
    else:
        
        major_std_1 = g2D_small.y_stddev.value
        
        if g2D_small.theta <= np.pi/2:
            theta_1 = np.pi/2 + g2D_small.theta.value
            
        elif g2D_small.theta > np.pi/2:
            theta_1 = g2D_small.theta.value - np.pi/2
             
        ecc_1 = np.sqrt(1.0 - (g2D_small.x_stddev.value/g2D_small.y_stddev.value)**2.0)
        
    return (KDE_bandwidth, KD_fit_sqresid, amp_0, xmean_0, ymean_0, major_std_0, theta_0,
            ecc_0, amp_1, xmean_1, ymean_1, major_std_1, theta_1, ecc_1)

In [5]:
def line(p, x):
    m, b = p
    return m*x + b

In [6]:
def linear_CMD_fit(x,y,xerr,yerr):
    """
    Does a linear fit to CMD data where x is color and y is amplitude, returning some fit 
    statistics
    
    Parameters
    ----------
    x : array-like
        color
    y : array-like
        magnitude
    xerr : array-like
        color errors
    yerr : array-like
        magnitude errors
    
    Returns
    -------
    slope : float
        slope of best-fit line
    r_squared : float
        Correlation coefficient (R^2)
    """
    
    data = RealData(x, y, sx=xerr, sy=yerr)
    
    mod = Model(line)
    
    odr = ODR(data, mod, beta0=[-0.1, np.mean(y)])
    out = odr.run()
    
    slope = out.beta[0]
    
    r_squared = r2_score(y, line(out.beta, x))
    
    return slope, r_squared

In [8]:
from celerite import terms
import emcee as mc

#In celerite-land, this is a DRW Kernel
class DRWTerm(terms.RealTerm):
    parameter_names = ("log_sigma", "log_tau")

    def get_real_coefficients(self, params):
        log_sigma, log_tau = params
        sigma = np.exp(log_sigma)
        tau = np.exp(log_tau)
        return (
            sigma**2.0 , 1/tau,
        )

def mean_per_visit(time,mag,err,dt_tol=100):
    """
    Calculates the mean per-visit point.
    
    Assume some delta time over which something is considered a separate visit.
    """
    visits = []
    visit = np.array([[time[0],mag[0],err[0]]])
    for i in range(1,len(time)):
        dif = time[i] - time[i-1]
        if dif <= dt_tol:
            visit = np.append(visit,[[time[i],mag[i],err[i]]],axis=0)
        else:
            visits.append(visit)
            visit = np.array([[time[i],mag[i],err[i]]])
    visits.append(visit)
    visits = np.array(visits)
    mean_times = []
    mean_mags = []
    mean_errs = []
    for visit in visits:
        mean_times.append(np.mean(visit[:,0]))
        mean_mags.append(np.mean(visit[:,1]))
        mean_errs.append(np.sqrt(np.sum(np.power(visit[:,2],2.0)))/len(visit))
    return np.array(mean_times),np.array(mean_mags),np.array(mean_errs)


def coarse_DRW(times,mags,errs):
    """
    Does a quick DRW fit using celerite+emcee to the average points in the visit. NOT GOOD FOR SHORT TERM VARIABILITY
    
    Parameters
    ----------
    times : array-like
    mags : array-like
    errs : array-like
    
    Returns
    -------
    DRW_sigma : float
    DRW_tau : float
    DRW_mean : float
    """
    
    mt,mm,me = mean_per_visit(times,mags,errs)
    
    #Bounds on sigma: the minimum error -> 10 times the range of mags
    #Bounds on tau: 0.25 times the minimum time difference -> 2 times the time baseline
    
    DRWbounds = dict(log_sigma=(np.log(np.min(me)), np.log(10.0*np.ptp(mm))), 
                 log_tau=(np.log(0.25*np.min(np.diff(mt))), np.log(2.0*np.ptp(mt))))
    
    #First guess on sigma: STD of points
    #First guess on tau: 0.5 * the time baseline
    kern = DRWTerm(log_sigma=np.std(mm), log_tau=np.log(0.5*np.ptp(mt)),bounds=DRWbounds)
    
    #Define and first compute of the DRW
    gp = celerite.GP(DRW_kern, mean=np.mean(mm), fit_mean = True)
    gp.compute(mt, me)
    
    #maximize likelihood, which requires autograd's numpy?
    initial_params = gp.get_parameter_vector()
    bounds = gp.get_parameter_bounds()
    import autograd.numpy as np
    soln = minimize(neg_log_like, initial_params, jac=grad_neg_log_like,
                method="L-BFGS-B", bounds=bounds, args=(mm, gp))
    import numpy as np
    
    #Now for the emceee
    def log_probability(params):
        gp.set_parameter_vector(params)
        lp = gp.log_prior()
        if not np.isfinite(lp):
            return -np.inf
        return gp.log_likelihood(mm) + lp
    
    #Initialize walkers
    initial = np.array(soln.x)
    ndim, nwalkers = len(initial), 32
    sampler = mc.EnsembleSampler(nwalkers, ndim, log_probability)
    
    #Burn in for 500 steps
    p0 = initial + 1e-8 * np.random.randn(nwalkers, ndim)
    p0, lp, _ = sampler.run_mcmc(p0, 500)
    
    #Reset, randomize the seed
    sampler.reset()
    np.random.seed(42)
    sampler.run_mcmc(p0, 3000)
    
    #flatten along step axis
    samples = sampler.flatchain
    DRW_sigma = np.exp(np.mean(samples[:,0]))
    DRW_tau = np.exp(np.mean(samples[:,1]))
    DRW_mean = np.mean(samples[:,2])
    
    return DRW_sigma,DRW_tau,DRW_mean

ImportError: No module named celerite